In [1]:
import cv2 as cv # opencv
import tensorflow as tf #tensorflow
import glob # to read files
import numpy as np

In [2]:
dirImage = 'jaffe/'
filenames = []
filenames += glob.glob(dirImage+"/*"+".tiff")
# print("Processing " + str(len(filenames)) + " images")
# print(filenames)

In [3]:
images = []
for file in filenames:
    img = np.asarray(cv.imread(file, 0))
    images.append(img)
images = np.asarray(images)
# print(images)

In [4]:
labels_count = 7
TRAINING_SIZE = 170
VALIDATION_SIZE = len(images) - TRAINING_SIZE
train_images = images[:TRAINING_SIZE, :, :] # 170, 256, 256
test_images = images[TRAINING_SIZE:, :, :]

In [5]:
label_file = open('temp_labels.txt');
label_data = label_file.read()
labels = []
for i in xrange(len(label_data)):
    if (label_data[i]!='\n'):
        labels.append(float(label_data[i]))
train_labels = np.asarray(labels[:TRAINING_SIZE], dtype=int)
test_labels = np.asarray(labels[170:], dtype=int)

In [6]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = len(labels_dense)
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    # print(type(np.int_(index_offset + labels_dense.ravel())))
    labels_one_hot.flat[np.int_(index_offset + labels_dense.ravel())] = 1
    return labels_one_hot

In [7]:
train_labels = dense_to_one_hot(train_labels.ravel(), labels_count)
# labels = labels.astype(np.uint8)
test_labels = dense_to_one_hot(test_labels.ravel(), labels_count)

In [8]:
train_images = train_images.reshape(TRAINING_SIZE, -1)
test_images = test_images.reshape(VALIDATION_SIZE, -1)

In [9]:
train_image_pixels = train_images.shape[1]
print 'Flat pixel values is %d'%(train_image_pixels)

Flat pixel values is 65536


In [10]:
train_image_width = np.int_(np.ceil(np.sqrt(train_image_pixels)))
train_image_height = np.int_(np.ceil(np.sqrt(train_image_pixels)))

In [11]:
print(train_image_width)
print(train_image_height)

256
256


# Build TF CNN model

In [12]:
def new_conv_layer(input_data, num_input_channels, num_filters, filter_shape, pool_shape, name):
    # setup input filter state
    conv_filt_shape = [filter_shape[0], filter_shape[1], num_input_channels, num_filters]
    
    # initialise weights and bias
    weights = tf.Variable(tf.truncated_normal(conv_filt_shape, stddev=0.03), name=name+'_W')
    bias = tf.Variable(tf.truncated_normal([num_filters]), name=name+'_b')

    # setup convolutional layer 
    out_layer = tf.nn.conv2d(input_data, weights, [1, 1, 1, 1], padding='SAME')
    out_layer += bias
    
    # applying relu non-linear activation
    out_layer = tf.nn.relu(out_layer)
    
    # performing max pooling
    ksize = [1, 2, 2, 1]
    strides = [1, 2, 2, 1]
    out_layer = tf.nn.max_pool(out_layer, ksize=ksize, strides=strides, padding='SAME')
    
    return out_layer

In [13]:
# input & output of NN

# images
x = tf.placeholder(tf.float32, [None, train_image_width*train_image_height])
# dynamically reshaping input
x_shaped = tf.reshape(x, [-1, train_image_width, train_image_height, 1])
# labels
y = tf.placeholder(tf.float32, [None, labels_count])

In [14]:
# creating sparse layers of CNN
layer1 = new_conv_layer(x_shaped, 1, 32, [5, 5], [2, 2], name='layer1')
layer2 = new_conv_layer(layer1, 32, 64, [5, 5], [2, 2], name='layer2')
layer3 = new_conv_layer(layer2, 64, 128, [5, 5], [2, 2], name='layer3')
layer4 = new_conv_layer(layer3, 128, 256, [5, 5], [2, 2], name='layer4')
flattened = tf.reshape(layer4, [-1, 16 * 16 * 256])

In [15]:
# calculating dense layers of CNN
wd1 = tf.Variable(tf.truncated_normal([16 * 16 * 256, 1000], stddev=0.03), name='wd1')
bd1 = tf.Variable(tf.truncated_normal([1000], stddev=0.01), name='bd1')
dense_layer1 = tf.matmul(flattened, wd1) + bd1
dense_layer1 = tf.nn.relu(dense_layer1)

# dropout for reducing overfitting
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(dense_layer1, keep_prob)

# another layer for softmax calculation and readout
wd2 = tf.Variable(tf.truncated_normal([1000, labels_count], stddev=0.03), name='wd2')
bd2 = tf.Variable(tf.truncated_normal([labels_count], stddev=0.01), name='bd2')
dense_layer2 = tf.matmul(dense_layer1, wd2) + bd2
y_ = tf.nn.softmax(dense_layer2)

In [16]:
# cross entropy cost function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=dense_layer2, labels=y))

# Training CNN

In [17]:
# set to 3000 iterations 
epochs = 10
DROPOUT = 0.5
batch_size = 10

In [18]:
# settings
learning_rate = 1e-5

In [20]:
# adding optimiser
optimiser = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

# defining accuracy assessment operation
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# setting up the initialisation operator
init_op = tf.global_variables_initializer()

# recording variable to store accuracy
tf.summary.scalar('accuracy', accuracy)

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('logs')

with tf.Session() as sess:
    # initialising variables
    sess.run(init_op)
    total_batch = int(len(train_labels) / batch_size)
    for epoch in xrange(epochs):
        avg_cost = 0
        batch_index = 0
        for j in xrange(total_batch):
            train_batch_images = train_images[batch_index:(batch_index+batch_size), :]
            train_batch_labels = train_labels[batch_index:(batch_index+batch_size), :]
            _, c = sess.run([optimiser, cross_entropy], feed_dict={x:train_batch_images, y:train_batch_labels, keep_prob:0.5})
            avg_cost += c
            batch_index += batch_size
        train_acc = sess.run(accuracy, feed_dict={x:train_images, y:train_labels})
        test_acc = sess.run(accuracy, feed_dict={x: test_images, y: test_labels})
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost), "train accuracy: {:.3f}".format(train_acc), "test accuracy: {:.3f}".format(test_acc))
        summary = sess.run(merged, feed_dict={x: test_images, y: test_labels})
        writer.add_summary(summary, epoch)
    print("\nTraining complete!")
    writer.add_graph(sess.graph)
    print(sess.run(accuracy, feed_dict={x: test_images, y: test_labels}))

('Epoch:', 1, 'cost =', '293.017', 'train accuracy: 0.318', 'test accuracy: 0.209')
('Epoch:', 2, 'cost =', '133.675', 'train accuracy: 0.335', 'test accuracy: 0.302')
('Epoch:', 3, 'cost =', '143.813', 'train accuracy: 0.229', 'test accuracy: 0.186')
('Epoch:', 4, 'cost =', '164.441', 'train accuracy: 0.459', 'test accuracy: 0.372')
('Epoch:', 5, 'cost =', '120.500', 'train accuracy: 0.382', 'test accuracy: 0.302')
('Epoch:', 6, 'cost =', '81.784', 'train accuracy: 0.406', 'test accuracy: 0.326')
('Epoch:', 7, 'cost =', '95.721', 'train accuracy: 0.429', 'test accuracy: 0.349')
('Epoch:', 8, 'cost =', '86.154', 'train accuracy: 0.388', 'test accuracy: 0.349')
('Epoch:', 9, 'cost =', '92.828', 'train accuracy: 0.482', 'test accuracy: 0.349')
('Epoch:', 10, 'cost =', '100.317', 'train accuracy: 0.465', 'test accuracy: 0.326')

Training complete!
0.325581
